In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import Compose, PILToTensor, Resize, Normalize
from PIL import Image
import os
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

True
1
0
NVIDIA GeForce RTX 3050 6GB Laptop GPU
Using device: cuda

NVIDIA GeForce RTX 3050 6GB Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
from torchvision.transforms import Compose, ToTensor, Resize, Normalize
cat_folder = './PetImages/Cat'
dog_folder = './PetImages/Dog'

transform = Compose([
    Resize((256, 256)),
    ToTensor(),
    Normalize(mean=(0.5,), std=(0.5,))
])

In [5]:
class PetDataset(Dataset):
    def __init__(self, folder, label, transform=None):
        self.folder = folder
        self.label = label
        self.transform = transform
        self.images = [os.path.join(folder, file) for file in os.listdir(folder) if file.endswith('.jpg') or file.endswith('.png')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, self.label
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            return None

# dataset
cat_dataset = PetDataset(cat_folder, label=0, transform=transform)
dog_dataset = PetDataset(dog_folder, label=1, transform=transform)

# dataloader
full_dataset = cat_dataset + dog_dataset
train_loader = DataLoader(full_dataset, batch_size=32, shuffle=True)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Input: (3, 256, 256), Output: (32, 256, 256)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Output: (64, 128, 128)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)  # Output: (128, 64, 64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Halves the dimensions
        self.flatten = nn.Flatten()
        self.fc1_input_size = 128 * (256 // 2 // 2 // 2) * (256 // 2 // 2 // 2)
        self.fc1 = nn.Linear(self.fc1_input_size, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)  # Output: (32, 128, 128)

        x = F.relu(self.conv2(x))
        x = self.pool(x)  # Output: (64, 64, 64)

        x = F.relu(self.conv3(x))
        x = self.pool(x)  # Output: (128, 32, 32)

        x = self.flatten(x)  # Flatten to (batch_size, 128 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)


In [7]:
criterion= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)
        
        optimizer.zero_grad()  # Reset gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")


e:\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1, Loss: 0.6009597917015735
Epoch 2, Loss: 0.4453740326448893
Epoch 3, Loss: 0.326827213798578
Epoch 4, Loss: 0.1823820491369145
Epoch 5, Loss: 0.07069133189715779
Epoch 6, Loss: 0.03849659207309089
Epoch 7, Loss: 0.02690229140614228
Epoch 8, Loss: 0.02880248682992645
Epoch 9, Loss: 0.017933116875755307
Epoch 10, Loss: 0.02456670050332599


In [9]:
def calculate_accuracy(model, data_loader, device):
    model.eval()  
    correct = 0
    total = 0

    with torch.no_grad():  
        for images, labels in data_loader:
            images = images.to(device, dtype=torch.float)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

train_accuracy = calculate_accuracy(model, train_loader, device)
print(f"Training Accuracy: {train_accuracy:.2f}%")

Training Accuracy: 99.72%
